In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
# Read the image
image = cv2.imread('images\\sheets\\example_sheet.png')

# Converts bgr to rgb color scale, default reads as bgr why?
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Uses Otsu method for thresholding the image 
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (80,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# TEST
# for c in cnts:
#     cv2.drawContours(image, [c], -1, (36,255,12), 2)

cnts_y_averages = []
for cnt in cnts:
    temp_total = 0
    for pnt in cnt:
        temp_total += pnt[0][1]
    cnts_y_averages.append(temp_total / len(cnt))
cnts_y_averages.sort()

filtered_cnts_y_averages = [cnts_y_averages[0]]
for i in range(1, len(cnts_y_averages)):
    if cnts_y_averages[i] > cnts_y_averages[i - 1] + 20: # offset
        filtered_cnts_y_averages.append(cnts_y_averages[i])

sliced_images = []
for i in range(1, len(filtered_cnts_y_averages)):
    temp_sliced_image = image[int(filtered_cnts_y_averages[i - 1]) : int(filtered_cnts_y_averages[i]), : len(image[0])]
    temp_sliced_image = cv2.copyMakeBorder(temp_sliced_image, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    sliced_images.append(temp_sliced_image)

# TEST
# Plotting all the images
fig, axes = plt.subplots(nrows=len(sliced_images), ncols=1, figsize=(14, 7))
for ax, img in zip(axes, sliced_images):
    ax.imshow(img)
    ax.axis('off')
plt.show()

# TEST
# Display the original image with detected lines
# cv2.imshow('Detected lines', image)
# cv2.imwrite('test.png', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
sliced_image_gray = cv2.cvtColor(sliced_images[3], cv2.COLOR_RGB2GRAY)

# Uses Otsu method for thresholding the image 
ret, sliced_image_thresh = cv2.threshold(sliced_image_gray, 0, 255, cv2.THRESH_OTSU)

ctrs, _ = cv2.findContours(sliced_image_thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

# Calculate average bounding contour area
areas = [cv2.contourArea(ctr) for ctr in ctrs]
average_area = sum(areas) / len(areas)

# Define thresholds as a percentage of the average area
lower_threshold = 30000  # 0.5 * average_area
upper_threshold = 40000 # 6.0 * average_area

extracted_move_boxes = []
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)
    area = w*h
    if lower_threshold < area < upper_threshold:
        # rect = cv2.rectangle(chmove_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.imshow('rect', rect)
        extracted_move_boxes.append(sliced_images[3][y:y + h, x:x + w])

# TEST
# Plotting all the images
fig, axes = plt.subplots(nrows=1, ncols=len(extracted_move_boxes), figsize=(7, 7))
for ax, img in zip(axes, extracted_move_boxes):
    ax.imshow(img)
    ax.axis('on')
plt.show()

In [ ]:
extracted_move_box_gray = cv2.cvtColor(extracted_move_boxes[0], cv2.COLOR_RGB2GRAY)

# Uses Otsu method for thresholding the image 
ret, extracted_move_box_thresh = cv2.threshold(extracted_move_box_gray, 0, 255, cv2.THRESH_OTSU)

ctrs, _ = cv2.findContours(extracted_move_box_thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

# Calculate average bounding contour area
areas = [cv2.contourArea(ctr) for ctr in ctrs]
average_area = sum(areas) / len(areas)

# Define thresholds as a percentage of the average area
lower_threshold = 625  # 0.5 * average_area
upper_threshold = 6400 # 6.0 * average_area

extracted_symbols = []
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)
    area = w*h
    if lower_threshold < area < upper_threshold:
        # rect = cv2.rectangle(chmove_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.imshow('rect', rect)
        extracted_symbols.append(extracted_move_box_thresh[y:y + h, x:x + w])

# Load model
model = tf.keras.models.load_model("..\\models\\channo_v0.4.keras")

# Define result list
result = []

# TEST
filtered_images = []

for extracted_symbol in extracted_symbols:
    # Define the desired size of the square image (AxA)
    desired_size = max(extracted_symbol.shape) + 15  # offset

    # Create a blank square image of the desired size
    resized_image = np.ones((desired_size, desired_size), dtype=np.uint8) * 255

    # Calculate the position to place the original image in the center
    x_offset = (desired_size - extracted_symbol.shape[1]) // 2
    y_offset = (desired_size - extracted_symbol.shape[0]) // 2

    # Place the original image in the center of the blank square image
    resized_image[y_offset:y_offset + extracted_symbol.shape[0], x_offset:x_offset + extracted_symbol.shape[1]] = extracted_symbol

    # Downscale the resized image to the target size (e.g., 28x28)
    downscaled_image = cv2.resize(resized_image, (28, 28))

    # Invert, normalize and reshape image to give input our model
    filtered_image = 255 - downscaled_image             # Invert

    # TEST
    # filtered_images.append(cv2.GaussianBlur(filtered_image, (5, 5), 0))
    filtered_images.append(filtered_image)
    
    filtered_image = filtered_image / 255.0             # Normalize
    filtered_image = filtered_image.reshape(1, 28, 28)  # Reshape

    # Pass filtered image to our model
    predictions = model.predict(filtered_image)
    predicted_class = np.argmax(predictions[0])
    print(predictions)
    result.append(predicted_class)

# TEST
# Plotting all the images
fig, axes = plt.subplots(nrows=1, ncols=len(filtered_images), figsize=(7, 7))
for ax, img in zip(axes, filtered_images):
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.show()

print(result)